In [1]:
import pandas as pd
import re
df_root = pd.read_csv('C:/Users/lui-m/Documents/GitHub/DS_Notebooks/ComunidadeDS/FTC_Python/datasets/train.csv')
df_root.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [3]:
# Fazendo cópia do dataframe lido
df = df_root.copy()

# Removendo espaço da string sem o loop for
df.loc[:, 'ID'] = df.loc[:, 'ID'].str.strip()
df.loc[:, 'Delivery_person_ID'] = df.loc[:, 'Delivery_person_ID'].str.strip()
df.loc[:, 'Road_traffic_density'] = df.loc[:, 'Road_traffic_density'].str.strip()
df.loc[:, 'Type_of_order'] = df.loc[:, 'Type_of_order'].str.strip()
df.loc[:, 'Type_of_vehicle'] = df.loc[:, 'Type_of_vehicle'].str.strip()

# Excluindo as linhas com a idade dos entregadores vazia
linhas_nao_vazias = df['Delivery_person_Age'] != 'NaN '
df = df.loc[linhas_nao_vazias, :]

# Conversao de texto/categoria/string para numeros inteiros
df['Delivery_person_Age'] = df['Delivery_person_Age'].astype(int)

# Conversao de texto/categoria/string para numeros decimais
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype(float)

# Conversao de texto para data
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

# Remove as linhas da coluna multiple_deliveries que tenham o conteúdo igual a 'NqN ' e fazendo o cast para inteiro
linhas_nao_vazias = df['multiple_deliveries'] != 'NaN '
df = df.loc[linhas_nao_vazias, :]
df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)

# Removendo texto de números e resetando index
df = df.reset_index(drop=True)
for i in range(len(df)):
    df.loc[i, 'Time_taken(min)'] = re.findall(r'\d+', df.loc[i, 'Time_taken(min)'])

### Quantidade de Entregas por Cidade

In [8]:
cols = ['ID', 'City']
group = df.loc[:, cols].groupby(['City']).count().reset_index()
group

,City,ID
0,Metropolitian,32031
1,NaN,1105
2,Semi-Urban,152
3,Urban,9517


### Quantidade de pessoas por idade

In [12]:
cols = ['ID', 'Delivery_person_Age']
age_group =df.loc[:, cols].groupby(['Delivery_person_Age']).count().reset_index()
age_group

,Delivery_person_Age,ID
0,15,38
1,20,2085
2,21,2109
3,22,2145
4,23,2039
5,24,2153
6,25,2117
7,26,2103
8,27,2100
9,28,2136


### Média de idade por Cidade

In [17]:
cols = ['Delivery_person_Age', 'City'] # Idade para aplicar a operação e qual a coluna vai lidar com essa operação

age_group = df.loc[:, cols].groupby('City').mean()
age_group

,Delivery_person_Age
City,
Metropolitian,29.792763
NaN,28.810860
Semi-Urban,32.565789
Urban,28.908480


### Media de Idade por Cidade e por Tipo de Veículo

In [30]:
cols = ['Delivery_person_Age', 'City', 'Type_of_vehicle']

age_vehicle_group = df.loc[:, cols].groupby(['City', 'Type_of_vehicle']).mean()
age_vehicle_group

Delivery_person_Age
City           Type_of_vehicle                      
Metropolitian  bicycle                     31.333333
               electric_scooter            29.961390
               motorcycle                  29.758281
               scooter                     29.811440
NaN            electric_scooter            29.579545
               motorcycle                  28.882960
               scooter                     28.559633
Semi-Urban     electric_scooter            36.000000
               motorcycle                  32.378571
               scooter                     34.333333
Urban          electric_scooter            28.995043
               motorcycle                  28.928302
               scooter                     28.857185